# Example of running the particle_tracking class and its associated functions for Single Particle Tracking

This notebook provides an example for using the particle_tracking class. It is implemented on .nd2 microscopy images which were acquired using the JOBS module in NIS elements (Nikon / LIM). Every channel (e.g., Phase contrast, GFP etc) is save in a separate .nd2 file, and the stream acquisition of the particle fluorescence is also saved separately. The cells were segmented using a Unet (see Mäkelä, Papagiannakis *et al*,  https://doi.org/10.7554/eLife.97465.3) which stored the cell labels in .tif images. The particle_tracking class has been adapted to work with the respective data formats. 

Author: Alexandros Papagiannakis, HHMI at Stanford University, 2025

## Generate file paths

The code below can be used to generate the file paths that will be used as variables to initialize the particle_tracking class. These file paths are specific to the JOBS protocols applied to automate image and stream acquisition. 

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append(os.path.join(os.path.abspath(os.getcwd()), "/particle_tracking_UNET_for_GitHub.py"))
import particle_tracking_UNET_for_GitHub as spt_gh

save_path = '/Users/alexandros/Downloads/test_data_466'
images_path = '/Volumes/Data_01/Alex Papagiannakis/Microscopy/muNS/HDR_sensor/50percent_light_300frames/06132021_CJW6723_30oC_GlyCAAT_50uM/20210613_203659_466'
masks_path = '/Volumes/Data_01/Alex Papagiannakis/Microscopy/muNS/HDR_sensor/50percent_light_300frames/06132021_CJW6723_30oC_GlyCAAT_50uM/06132021_phase_tif/20210613_203659_466_tif'
stream_channel = 'FITC'
phase_channel = 'Trans'
signal_channel = 'mCherry'
interval = 50
images_names = os.listdir(images_path)
masks_names = os.listdir(masks_path)
positions = list(range(0,50))
experiment = '20210613_203659_466'

position = 0
# This script iterates over the JOBS particle tracking output, and the Unet labeled masks
if position <= 9:
    position_string = 'Point000'+str(position)
elif position > 9:
    position_string = 'Point00'+str(position) # the JOBS xy positions are defined as Point
# get the matching Unet path
matching = [s for s in masks_names if position_string in s]
matching = [s for s in matching if 'mask' in s][0]
unet_path = masks_path + '/' + matching
# get the matching stream images and snapshots
matching = [s for s in images_names if position_string in s]
fast_time_lapse_path = images_path + '/' + matching[0] 
snapshots_paths = [images_path + '/' + matching[2], images_path + '/' + matching[3], images_path + '/' + matching[1]]

IndentationError: expected an indented block after 'elif' statement on line 104 (particle_segmentation.py, line 107)

## Class initialization
The image paths above, together with the experiment and CY position IDs will be used to initialize the class and store it in a variable called "particle"

In [ ]:
particle = spt_gh.particle_tracking(unet_path, snapshots_paths, fast_time_lapse_path, experiment, position, interval, save_path) #initialize the particle tracking class


## Check cell segmentation
The Unet cell segmentation will be checked. This step is also important to confirm that the Unet masks have been successfully incorporated in the class to match the phase contrast image. 

In [ ]:
particle.show_unet_masks() # visualize the cell segmentation

## Test the particle segmentation parameters
This interactive function can be used to test the particle segmentation parameters.

In [4]:
# particle.test_segmentation_parameters(10, post_process=True)
# The input() function that is used here does not work in some Jupyter notebooks.
# This function should work in any other full-Python kernel.

## Segment the particles
The particle segmentation method is applied for 50 frames (since this is a test) using the following parameters:
1. log_adaptive_parameters=[4, 1000, 99, 2, 9, -6, 0]
2. min_particle_size=3
3. max_particle_size=60
4. min_particle_aspect_ratio=0.3
5. post_processing_threshold=90
6. box_size=7

In [5]:
particle_df = particle.getting_the_particles(channel_offset=(0,0), log_adaptive_parameters=[4, 1000, 99, 2, 9, -6, 0], 
                                      min_particle_size=3, max_particle_size=60, min_particle_aspect_ratio=0.3,
                                      post_processing_threshold=90, box_size=7, analysis_range=(0,particle.n_frames), 
                                      metric='raw pixels', operation='sum',  gaussian_fit_show = False) # particle segmentation

In [6]:
particle_df.columns.tolist()

## Particle tracking
The stored particle_df Pandas DataFrame will be used to track the particles. Merged trajectories will be removed, the trajectories that have a length less than 1.7% of the total stream acquisition length in frames will also be removed, and non-overlapping trajectories that belong to the same cell will be merged to reconstruct maximum length trajectories. 

In [ ]:
particle_tracking_df = particle.run_particle_tracking(max_radius=8, memory=3, fluorescence_bandpass=(0.2,5), fraction_length=0.017, merged=True, cell_connect=True) # particle tracking